<a href="https://colab.research.google.com/github/nick0410/melody-cheif/blob/master/recommendations_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd

In [22]:
df=pd.read_excel("/content/hackathon_dataset.xlsx")
df.head()

,id,name,genre,location,skill_level,instr1,instr2,profile_link,followers,tracks,availability
0,1,Aarav Patel,Indie Folk,Mumbai,Advanced,Guitar,Vocals,https://soundcloud.com/aarav,2000,15,Yes
1,2,Neha Sharma,Classical,Delhi,Intermediate,Violin,Piano,https://soundcloud.com/neha,1500,10,No
2,3,Raj Singh,Hip-Hop,Bangalore,Beginner,Rapping,Beatboxing,https://soundcloud.com/raj,800,8,Yes
3,4,Priya Verma,Pop,Hyderabad,Advanced,Vocals,Guitar,https://soundcloud.com/priya,3000,20,Yes
4,5,Vikram Rao,Rock,Chennai,Intermediate,Electric Guitar,NaN,https://soundcloud.com/vikram,1200,12,No


In [23]:
import numpy as np

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
user_data = {}
user_item_matrix = pd.DataFrame()
# (1)

In [26]:
def add_user(user_id):
    user_data[user_id] = {}


In [27]:
user_ids = ['User1', 'User2', 'User3', 'User4', 'User5']
for user_id in user_ids:
    add_user(user_id)
# 2
ratings = np.random.randint(1, 6, size=(len(user_ids), len(df)))
user_item_matrix = pd.DataFrame(ratings, columns=df['name'], index=user_ids)
# 3

In [28]:
def content_based_recommendations(df, instr=None):
    if instr:
        df = df[(df['instr1'].str.contains(instr, case=False, na=False)) |
                 (df['instr2'].str.contains(instr, case=False, na=False))]
    return df
# 4

In [29]:
def collaborative_filtering_recommendations(user_item_matrix, user_id):
    similarity_matrix = cosine_similarity(user_item_matrix)
    similarity_df = pd.DataFrame(similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)
    user_similarities = similarity_df[user_id]
    similar_users = user_similarities.sort_values(ascending=False).index[1:]
    recommended_musicians = pd.DataFrame(columns=df.columns)

    for similar_user in similar_users:
        similar_user_ratings = user_item_matrix.loc[similar_user]
        recommended_items = similar_user_ratings[similar_user_ratings >= 4]

        for musician in recommended_items.index:
            if musician not in recommended_musicians['name'].values:
                musician_data = df[df['name'] == musician]
                recommended_musicians = pd.concat([recommended_musicians, musician_data])
    return recommended_musicians
# 4

In [30]:
def hybrid_recommendations(df, user_item_matrix, user_ids, instr=None):
    all_recommendations = pd.DataFrame()
    for user_id in user_ids:
        content_recs = content_based_recommendations(df, instr)
        collab_recs = collaborative_filtering_recommendations(user_item_matrix, user_id)
        hybrid_recs = pd.concat([content_recs, collab_recs]).drop_duplicates().reset_index(drop=True)
        hybrid_recs = hybrid_recs[(hybrid_recs['instr1'].str.contains(instr, case=False, na=False)) |
                                   (hybrid_recs['instr2'].str.contains(instr, case=False, na=False))]
        hybrid_recs['user_id'] = user_id
        all_recommendations = pd.concat([all_recommendations, hybrid_recs])
    return all_recommendations.reset_index(drop=True)
# 5

this is just for example - erase this if required.

In [32]:
user_ids = ['User1']
hybrid_result = hybrid_recommendations(df, user_item_matrix, user_ids, instr="Synthesizers")
print(hybrid_result[['name', 'genre', 'skill_level', 'location', 'instr1', 'instr2', 'availability', 'profile_link']])


              name       genre   skill_level   location        instr1  \
0        Sara Khan  Electronic      Beginner  Bangalore  Synthesizers   
1      Shruti Nair  Electronic      Advanced     Mumbai  Synthesizers   
2     Puneet Singh         Pop  Intermediate     Mumbai        Vocals   
3     Sakshi Reddy         Pop  Intermediate  Bangalore        Vocals   
4      Kavya Singh         Pop  Intermediate       Pune        Vocals   
5  Siddharth Mehta         Pop      Advanced  Bangalore        Vocals   

          instr2 availability                            profile_link  
0         Vocals          Yes             https://soundcloud.com/sara  
1         Vocals          Yes           https://soundcloud.com/shruti  
2   Synthesizers           No           https://soundcloud.com/puneet  
3   Synthesizers           No           https://soundcloud.com/sakshi  
4   Synthesizers           No            https://soundcloud.com/kavya  
5   Synthesizers          Yes  https://soundcloud.com/si